# AlphaScore NoteBook

## Kenya Data

#### Importing Packages

In [ ]:
import pyodbc
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sn
from datetime import datetime
from matplotlib.pyplot import figure
import csv
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext
import os

#### Connecting Database

In [ ]:
server = 'EQDB02Z.database.windows.net'
database = 'SUGAR'
username = #add your username
password = #add your password
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

#### The Query

In [ ]:
#Change to the date before run
query = f'''
declare @dates varchar(50) = '2023-10-03';

with temp1_reduction as (
select  l.primary_email_address,
        l.id as leadID,
        l.name,
        l.status,
        l.broker_id, 
        l.date_modified,
		cast(l.date_entered as date) as LdataCreated,
        u.user_name, 
        l.account_id, 
        a.id, 
        a.account_status,
        a.application_status,
        a.trading_status, 
        cast(l.date_entered as date) as ldatecreation,
        @dates as todays,
        DATEDIFF(day,cast(l.date_entered as date),@dates) as datediffs,
    case when (l.status like 'New' and DATEDIFF(day,cast(l.date_entered as date),@dates) = 0) then 2 
        when (l.status like 'New' and DATEDIFF(day,cast(l.date_entered as date),@dates) >= 1) then 3
        when (l.status like 'Attempted1' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  0 and 2) then 1
        when (l.status like 'Attempted1' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  3 and 5) then 2
        when (l.status like 'Attempted1' and DATEDIFF(day,cast(l.date_entered as date),@dates) > 5) then 3
        when (l.status like 'Attempted2' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  0 and 2) then 1
        when (l.status like 'Attempted2' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  3 and 5) then 1.5
        when (l.status like 'Attempted2' and DATEDIFF(day,cast(l.date_entered as date),@dates) > 5) then 2
        when (l.status like 'Attempted3' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  0 and 2) then 0.5
        when (l.status like 'Attempted3' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  3 and 5) then 1
        when (l.status like 'Attempted3' and DATEDIFF(day,cast(l.date_entered as date),@dates) > 5) then 1.5
        when (l.status like 'Cold' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  0 and 5) then 0
        when (l.status like 'Cold' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  6 and 10) then 0.5
        when (l.status like 'Cold' and DATEDIFF(day,cast(l.date_entered as date),@dates) > 10) then 1
        when (l.status like 'Warm' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  0 and 5) then 1
        when (l.status like 'Warm' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  6 and 10) then 2
        when (l.status like 'Warm' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  11 and 20) then 3
        when (l.status like 'Warm' and DATEDIFF(day,cast(l.date_entered as date),@dates) > 20) then 4
        when (l.status like 'Hot' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  0 and 2) then 1
        when (l.status like 'Hot' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  3 and 5) then 2
        when (l.status like 'Hot' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  6 and 14) then 3
        when (l.status like 'Hot' and DATEDIFF(day,cast(l.date_entered as date),@dates) > 14) then 4
        when (l.status in ('BadData','NotInterested','NotFit','NotAccepted','Test', 'Rejected','ArchivedNoInterest','Duplicate','Restricted')) then 0   
        else 4 end as L_Status_scoring,
    case when (a.application_status like 'Incomplete Application') then 1 
        when (a.application_status like 'Live Application') then 2
        when (a.application_status like 'Complete Application') then 3
        when (a.application_status like 'Declaration Required') then 4
        else 0 end as C_Status_scoring,
    case when (a.trading_status like 'NotFunded') then 6 
        when (a.trading_status like 'Funded') then 5
        else 0 end as C_TradingStatus_scoring,
    rank() over (partition by l.primary_email_address order by l.date_modified desc ) as ranks_depos  
from sugar.dbo.leads l
join sugar.dbo.brokers b on l.broker_id = b.id 
left join sugar.dbo.accounts a on l.account_id = a.id 
join sugar.dbo.users u on u.id = l.assigned_user_id 
where l.primary_email_address not like '%+%'
and l.primary_email_address not like '%test%' 
and l.date_entered > '2022-11-01'
and l.status not in ('Duplicate','Test','Restricted','NotFit')
and (a.trading_status is null or a.trading_status not in ('Active','Dormant','Inactive'))
)
,
temp1 as(
select * from temp1_reduction where temp1_reduction.ranks_depos = 1)
,
temp2_reduction as (
select  t.account_id,
        a.primary_email_address, 
        a.broker_id ,
        t.transaction_status, 
        max(t.date_entered) as LastDepo,
        case when (DATEDIFF(day,cast( max(t.date_entered) as date),@dates) < 2 and  a.trading_status like 'NotFunded' and t.name like 'Deposit' and t.transaction_status like 'Rejected') then 7        
        when (DATEDIFF(day,cast( max(t.date_entered)  as date),@dates) >= 2  and  a.trading_status like 'NotFunded' and t.name like 'Deposit' and t.transaction_status like 'Rejected') then 5
        when (a.trading_status like 'Funded' and t.name like 'Deposit' and t.transaction_status like 'Rejected') then 3
        else 0 end as Upweight_Depo_Rejection,
        rank() over (partition by a.primary_email_address order by t.date_entered desc ) as ranks_status
from sugar.dbo.transactions t
join sugar.dbo.accounts a on a.id = t.account_id 
where t.name like 'Deposit'
and year(t.date_entered)>2021
and a.classification like 'DirectClient'
and a.primary_email_address not like '%test%'
and a.primary_email_address not like '%+%'
and t.direction like 'In'
group by t.account_id,a.primary_email_address, a.broker_id , t.name,t.transaction_status, a.trading_status,t.date_entered
),
temp2 as(
select * from temp2_reduction where temp2_reduction.ranks_status = 1
)
,
temp3 as (
select a.primary_email_address,a.broker_id, 
        max(case when (ca.name like 'Login') then ca.date_entered else 0 end) as last_login,
        case 
            when ( datediff(day,max(case when (ca.name like 'Login') then ca.date_entered else 0 end),@dates) < 2 and 
            count(case when (ca.name like '%assword%') then 1 else 0 end) > 2 ) then 2.5
            when (count(case when (ca.name like '%assword%') then 1 else 0 end) > 2 ) then 2
            else 0 end as Loging_PasswordUpweight,
        datediff(day,max(case when (ca.name like 'Login') then ca.date_entered else 0 end),@dates) as difflastlogin, 
        count(case when (ca.name like '%assword%') then 1 else 0 end) as psschanges 
        from sugar.dbo.client_activities ca 
join sugar.dbo.accounts a on a.id = ca.account_id 
where a.classification like 'DirectClient'
and a.primary_email_address not like '%test%'
and a.primary_email_address not like '%+%'
group by a.primary_email_address,a.broker_id
),
temp4_reduction as(
select COALESCE (l.primary_email_address, a.primary_email_address) as primary_email_address,  c.broker_id, c.date_entered
	from sugar.dbo.calls c 
	left join sugar.dbo.leads l on c.parent_id = l.id 
	left join sugar.dbo.accounts a on c.parent_id = a.id 
	where year(c.date_entered)>=2022 
)
,
temp4 as(
select primary_email_address, broker_id, cast(max(date_entered) as date) as Last_call_date 
from temp4_reduction 
group by primary_email_address, broker_id)


select temp1.primary_email_address,
    temp1.user_name,
    temp1.leadid,
    b.name as Broker,
    temp1.status,
    case when (temp1.account_status like 'Active') then 'Approved' else temp1.account_status end as Client_status,
    temp1.application_status,
    temp1.trading_status,
    temp1.LdataCreated as Lead_created_date,
    cast(max(temp2.LastDepo) as date) as Last_depoist_Date,
    temp2.transaction_status,
    cast(temp3.last_login as date) as last_login,
    temp1.L_Status_scoring as Lead_Status_scoring,
    temp1.C_Status_scoring as Client_Status_scoring,
    temp1.C_TradingStatus_scoring as Trading_status_scoring,
    temp4.Last_call_date as Last_call_date,
    COALESCE (temp2.Upweight_Depo_Rejection,0) as Upweight_Deposit_Rejection,
    COALESCE (temp3.Loging_PasswordUpweight,0) as Loging_Password_Upweight,
    
    case when (temp4.Last_call_date is not null or datediff(day,cast(max(Last_call_date) as date),@dates) >=14) then 0.1 
    when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates) >=7 and datediff(day,cast(max(Last_call_date) as date),@dates) <14)) then 0.2  
    when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates) >=5 and datediff(day,cast(max(Last_call_date) as date),@dates) <7)) then 0.3  
    when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates) >=1 and datediff(day,cast(max(Last_call_date) as date),@dates) <5)) then 0.4  
    when (temp4.Last_call_date is not null or datediff(day,cast(max(Last_call_date) as date),@dates)  <1) then 0.5  
    else 1 end as Weight_last_call, 
    
    --sum(temp1.L_Status_scoring + temp1.C_Status_scoring + temp1.C_TradingStatus_scoring + COALESCE (temp2.Upweight_Depo_Rejection,0) + COALESCE(temp3.Loging_PasswordUpweight,0) ) as Final_score, 
    
    (sum(temp1.L_Status_scoring + temp1.C_Status_scoring + temp1.C_TradingStatus_scoring + COALESCE (temp2.Upweight_Depo_Rejection,0) + COALESCE(temp3.Loging_PasswordUpweight,0) ) *
    (case when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates) >=14)) then 0.9 -- last call more than 14 days ago
    when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates) >=7 and datediff(day,cast(max(Last_call_date) as date),@dates) <14)) then 0.8  -- last call between 7 to 14 days ago
    when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates) >=5 and datediff(day,cast(max(Last_call_date) as date),@dates) <7)) then 0.7  -- last call between 5 to 7 days ago
    when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates) >=1 and datediff(day,cast(max(Last_call_date) as date),@dates) <5)) then 0.6  -- last call between 1 to 5 days ago
    when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates)  <1)) then 0.5  -- last call within 24hrs
    else 1 end)) as Final_score_Call_Weighted
    from temp1 
left join temp2 on (temp1.primary_email_address = temp2.primary_email_address and temp1.broker_id = temp2.broker_id)
left join temp3 on (temp1.primary_email_address = temp3.primary_email_address and temp1.broker_id = temp3.broker_id)
left join temp4 on (temp1.primary_email_address = temp4.primary_email_address and temp1.broker_id = temp4.broker_id)
join sugar.dbo.brokers b on temp1.broker_id = b.id 
where b.name in ('FXPesa','EGMSecurities') 
group by 
    temp1.primary_email_address,
    temp1.user_name,
    temp1.leadid,
    b.name,
    temp1.status,
    temp1.account_status,
    temp1.application_status,
    temp1.trading_status,
    temp1.LdataCreated,
    temp2.LastDepo,
    temp2.transaction_status,
    temp3.last_login,
    temp1.L_Status_scoring,
    temp1.C_Status_scoring,
    temp1.C_TradingStatus_scoring,
    temp2.Upweight_Depo_Rejection,
    temp3.Loging_PasswordUpweight,
    temp4.Last_call_date
order by primary_email_address desc
        '''
ALPHA_KE = pd.read_sql(query, cnxn)

#### Aggregating Data

In [ ]:
options_ke = ['Allan.Kiprono','Davies.Nyaneko','Elizabeth.Mueni','emmanuel.agaton','fridah.gitonga','halima.naira',
    'Ivy.Chege','Jackline.Mutuma','Jecton.Onyango','jesse.malenya','lewis.martin','lilian.nyakundi','Mary.Jaoko',
    'Millicent.Nzenge','Rayan.Aziz','Samuel.Wanjiru','Susan.Thuranira','tabitha.kyalo','Yvonne.Orwaru',
    'Abdirashid.Shariff','Paul.Castro','Bonface.Akumu','Arwins.Rota','Kevin.Kimani', 'James.Wesonga', 'Norbert.Mashauri', 'Ritah.Kabanyoro', 'jared.nyamweya', 'stephen.kariuki', 'Zainab.Adan']

ALPHA_KE['CRMLink'] = "https://honey.equitiglobalmarkets.com/#/Leads/"+ALPHA_KE['leadid']
ALPHA_KE = ALPHA_KE.sort_values(['user_name', 'Final_score_Call_Weighted'],ascending = [True, False])

In [ ]:
rslt_df_KE = ALPHA_KE[ALPHA_KE['user_name'].isin(options_ke)]
rslt_df_KE_100 = rslt_df_KE.groupby('user_name').apply(lambda group: group.head(100)).reset_index(drop = True)

#### Exporting Files

In [ ]:
# Set path
path = r'Kenya'
path2 = r'Kenya'

# set date
date = datetime.now().strftime('%Y-%m-%d')

newpath = f'{path}\{date}' 
if not os.path.exists(newpath):
    os.makedirs(newpath)


# Groupby 'Name' column and export different csv's with names in manager
for x,y in rslt_df_KE_100.groupby('user_name'):
    y.to_excel(f'{newpath}\{x}_{date}.xlsx', index = False)
    
# Back up file 
ALPHA_KE.to_excel(f'{path2}\ALPHA_KE_{date}.xlsx', index = False)

## Jordan, SCA and SC Data

#### Jordan and SC Query

In [ ]:
#Change to the date before run
query = f'''
declare @dates varchar(50) = '2023-08-31'; 

with temp1_reduction as (
select  l.primary_email_address, 
		l.id as leadid,
        l.name,
        l.status,
        l.broker_id, 
        l.date_modified,
		cast(l.date_entered  as date) as LdataCreated,
        u.user_name, 
        l.account_id, 
        a.id, 
        a.account_status,
        a.application_status,
        a.trading_status, 
        cast(l.date_entered as date) as ldatecreation,
        @dates as todays,
        DATEDIFF(day,cast(l.date_entered as date),@dates) as datediffs,
    case when (l.status like 'New' and DATEDIFF(day,cast(l.date_entered as date),@dates) = 0) then 2 
        when (l.status like 'New' and DATEDIFF(day,cast(l.date_entered as date),@dates) >= 1) then 3
        when (l.status like 'Attempted1' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  0 and 2) then 1
        when (l.status like 'Attempted1' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  3 and 5) then 2
        when (l.status like 'Attempted1' and DATEDIFF(day,cast(l.date_entered as date),@dates) > 5) then 3
        when (l.status like 'Attempted2' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  0 and 2) then 1
        when (l.status like 'Attempted2' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  3 and 5) then 1.5
        when (l.status like 'Attempted2' and DATEDIFF(day,cast(l.date_entered as date),@dates) > 5) then 2
        when (l.status like 'Attempted3' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  0 and 2) then 0.5
        when (l.status like 'Attempted3' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  3 and 5) then 1
        when (l.status like 'Attempted3' and DATEDIFF(day,cast(l.date_entered as date),@dates) > 5) then 1.5
        when (l.status like 'Cold' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  0 and 5) then 0
        when (l.status like 'Cold' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  6 and 10) then 0.5
        when (l.status like 'Cold' and DATEDIFF(day,cast(l.date_entered as date),@dates) > 10) then 1
        when (l.status like 'Warm' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  0 and 5) then 1
        when (l.status like 'Warm' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  6 and 10) then 2
        when (l.status like 'Warm' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  11 and 20) then 3
        when (l.status like 'Warm' and DATEDIFF(day,cast(l.date_entered as date),@dates) > 20) then 4
        when (l.status like 'Hot' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  0 and 2) then 1
        when (l.status like 'Hot' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  3 and 5) then 2
        when (l.status like 'Hot' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  6 and 14) then 3
        when (l.status like 'Hot' and DATEDIFF(day,cast(l.date_entered as date),@dates) > 14) then 4
        when (l.status in ('BadData','NotInterested','NotFit','NotAccepted','Test', 'Rejected','ArchivedNoInterest','Duplicate','Restricted')) then 0   
        else 4 end as L_Status_scoring,
    case when (a.application_status like 'Incomplete Application') then 1 
        when (a.application_status like 'Live Application') then 2
        when (a.application_status like 'Complete Application') then 3
        when (a.application_status like 'Declaration Required') then 4
        else 0 end as C_Status_scoring,
    case when (a.trading_status like 'NotFunded') then 6 
        when (a.trading_status like 'Funded') then 5
        else 0 end as C_TradingStatus_scoring,
    rank() over (partition by l.primary_email_address order by l.date_modified desc ) as ranks_depos  
from sugar.dbo.leads l
join sugar.dbo.brokers b on l.broker_id = b.id 
left join sugar.dbo.accounts a on l.account_id = a.id 
join sugar.dbo.users u on u.id = l.assigned_user_id 
where l.primary_email_address not like '%+%'
and l.primary_email_address not like '%test%' 
and l.date_entered > '2022-11-01'
and l.status not in ('Duplicate','Test','Restricted','NotFit')
and (a.trading_status is null or a.trading_status not in ('Active','Dormant','Inactive'))
)
,
temp1 as(
select * from temp1_reduction where temp1_reduction.ranks_depos = 1)
,
temp2_reduction as (
select  t.account_id,a.primary_email_address, a.broker_id ,t.transaction_status, max(t.date_entered) as LastDepo,
        case when (DATEDIFF(day,cast( max(t.date_entered)  as date),@dates) < 2 and  a.trading_status like 'NotFunded' and t.name like 'Deposit' and t.transaction_status like 'Rejected') then 7        
        when (DATEDIFF(day,cast( max(t.date_entered)  as date),@dates) >= 2  and  a.trading_status like 'NotFunded' and t.name like 'Deposit' and t.transaction_status like 'Rejected') then 5
        when (a.trading_status like 'Funded' and t.name like 'Deposit' and t.transaction_status like 'Rejected') then 3
        else 0 end as Upweight_Depo_Rejection,
        rank() over (partition by a.primary_email_address order by t.date_entered desc ) as ranks_status
from sugar.dbo.transactions t
join sugar.dbo.accounts a on a.id = t.account_id 
where t.name like 'Deposit'
and year(t.date_entered)>2021
and a.classification like 'DirectClient'
and a.primary_email_address not like '%test%'
and a.primary_email_address not like '%+%'
and t.direction like 'In'
group by t.account_id,a.primary_email_address, a.broker_id , t.name,t.transaction_status, a.trading_status,t.date_entered
),
temp2 as(
select * from temp2_reduction where temp2_reduction.ranks_status = 1
)
,
temp3 as (
select a.primary_email_address,a.broker_id, 
        max(case when (ca.name like 'Login') then ca.date_entered else 0 end) as last_login,
        case when ( datediff(day,max(case when (ca.name like 'Login') then ca.date_entered else 0 end),@dates) < 2 and 
                     count(case when (ca.name like '%assword%') then 1 else 0 end) > 2 ) then 2.5
        when (count(case when (ca.name like '%assword%') then 1 else 0 end) > 2 ) then 2
        else 0 end as Loging_PasswordUpweight,
        datediff(day,max(case when (ca.name like 'Login') then ca.date_entered else 0 end),@dates) as difflastlogin, 
        count(case when (ca.name like '%assword%') then 1 else 0 end) as psschanges 
        from sugar.dbo.client_activities ca 
join sugar.dbo.accounts a on a.id = ca.account_id 
where a.classification like 'DirectClient'
and a.primary_email_address not like '%test%'
and a.primary_email_address not like '%+%'
group by a.primary_email_address,a.broker_id
),
temp4_reduction as(
select COALESCE (l.primary_email_address, a.primary_email_address) as primary_email_address,  c.broker_id, c.date_entered
	from sugar.dbo.calls c 
	left join sugar.dbo.leads l on c.parent_id = l.id 
	left join sugar.dbo.accounts a on c.parent_id = a.id 
	where year(c.date_entered)>=2022 
)
,
temp4 as(
select primary_email_address, broker_id, cast(max(date_entered) as date) as Last_call_date 
from temp4_reduction 
group by primary_email_address, broker_id)
,
temp5 as (
select temp1.primary_email_address,
    temp1.user_name,
    temp1.leadid,
    b.name,
    temp1.status,
    temp1.account_status,
    case when (temp1.account_status like 'Active') then 'Approved' else temp1.account_status end as Client_status,
    (sum(temp1.L_Status_scoring + temp1.C_Status_scoring + temp1.C_TradingStatus_scoring + COALESCE (temp2.Upweight_Depo_Rejection,0) + COALESCE(temp3.Loging_PasswordUpweight,0) ) *
    (case when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates) >=14)) then 0.9 -- last call more than 14 days ago
    when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates) >=7 and datediff(day,cast(max(Last_call_date) as date),@dates) <14)) then 0.8  -- last call between 7 to 14 days ago
    when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates) >=5 and datediff(day,cast(max(Last_call_date) as date),@dates) <7)) then 0.7  -- last call between 5 to 7 days ago
    when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates) >=1 and datediff(day,cast(max(Last_call_date) as date),@dates) <5)) then 0.6  -- last call between 1 to 5 days ago
    when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates)  <1)) then 0.5  -- last call within 24hrs
    else 1 end)) as Final_score_Call_Weighted
 from temp1 
left join temp2 on (temp1.primary_email_address = temp2.primary_email_address and temp1.broker_id = temp2.broker_id)
left join temp3 on (temp1.primary_email_address = temp3.primary_email_address and temp1.broker_id = temp3.broker_id)
left join temp4 on (temp1.primary_email_address = temp4.primary_email_address and temp1.broker_id = temp4.broker_id)
join sugar.dbo.brokers b on temp1.broker_id = b.id 
where b.name in ('Equiti JO','Equiti SC') 
group by 
    temp1.primary_email_address,
    temp1.user_name,
    temp1.leadid,
    b.name,
    temp1.status,
    temp1.account_status,
    temp1.application_status,
    temp1.trading_status,
    temp1.LdataCreated,
    temp2.LastDepo,
    temp2.transaction_status,
    temp3.last_login,
    temp1.L_Status_scoring,
    temp1.C_Status_scoring,
    temp1.C_TradingStatus_scoring,
    temp2.Upweight_Depo_Rejection,
    temp3.Loging_PasswordUpweight,
    temp4.Last_call_date
)
select temp5.primary_email_address as Email,
    temp5.leadid,
	temp5.user_name as Name,
    temp5.name as Broker,
    temp5.status as Lead_Status,
    temp5.Client_status,
    temp5.Final_score_Call_Weighted,
    case when (temp5.Final_score_Call_Weighted > 12.999) then 'Very High'
    	when (temp5.Final_score_Call_Weighted between 8 and 12.999) then 'High'
    	when (temp5.Final_score_Call_Weighted between 5 and 7.999) then 'Average'
    	when (temp5.Final_score_Call_Weighted between 3 and 4.999) then 'Low'
    	else 'Very Low'  end as FinalScore
from temp5    
where temp5.Client_status not like 'Closed'
order by temp5.Final_score_Call_Weighted desc
        '''
ALPHA_JOSC = pd.read_sql(query, cnxn)

#### SCA Query

In [ ]:
server = 'EQDB02Z.database.windows.net'
database = 'SUGAR-SCA'
username = # add your username
password = # add your password
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [ ]:
# extract data from lead and account tables
query = f'''
declare @dates varchar(50) = '2023-08-31';

with temp1_reduction as (
select  l.primary_email_address,
		l.id as leadid, 
        l.name,
        l.status,
        l.broker_id, 
        l.date_modified,
		cast(l.date_entered  as date) as LdataCreated,
        u.user_name, 
        l.account_id, 
        a.id, 
        a.account_status,
        a.application_status,
        a.trading_status, 
        cast(l.date_entered as date) as ldatecreation,
        @dates as todays,
        DATEDIFF(day,cast(l.date_entered as date),@dates) as datediffs,
    case when (l.status like 'New' and DATEDIFF(day,cast(l.date_entered as date),@dates) = 0) then 2 
        when (l.status like 'New' and DATEDIFF(day,cast(l.date_entered as date),@dates) >= 1) then 3
        when (l.status like 'Attempted1' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  0 and 2) then 1
        when (l.status like 'Attempted1' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  3 and 5) then 2
        when (l.status like 'Attempted1' and DATEDIFF(day,cast(l.date_entered as date),@dates) > 5) then 3
        when (l.status like 'Attempted2' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  0 and 2) then 1
        when (l.status like 'Attempted2' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  3 and 5) then 1.5
        when (l.status like 'Attempted2' and DATEDIFF(day,cast(l.date_entered as date),@dates) > 5) then 2
        when (l.status like 'Attempted3' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  0 and 2) then 0.5
        when (l.status like 'Attempted3' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  3 and 5) then 1
        when (l.status like 'Attempted3' and DATEDIFF(day,cast(l.date_entered as date),@dates) > 5) then 1.5
        when (l.status like 'Cold' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  0 and 5) then 0
        when (l.status like 'Cold' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  6 and 10) then 0.5
        when (l.status like 'Cold' and DATEDIFF(day,cast(l.date_entered as date),@dates) > 10) then 1
        when (l.status like 'Warm' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  0 and 5) then 1
        when (l.status like 'Warm' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  6 and 10) then 2
        when (l.status like 'Warm' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  11 and 20) then 3
        when (l.status like 'Warm' and DATEDIFF(day,cast(l.date_entered as date),@dates) > 20) then 4
        when (l.status like 'Hot' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  0 and 2) then 1
        when (l.status like 'Hot' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  3 and 5) then 2
        when (l.status like 'Hot' and DATEDIFF(day,cast(l.date_entered as date),@dates) BETWEEN  6 and 14) then 3
        when (l.status like 'Hot' and DATEDIFF(day,cast(l.date_entered as date),@dates) > 14) then 4
        when (l.status in ('BadData','NotInterested','NotFit','NotAccepted','Test', 'Rejected','ArchivedNoInterest','Duplicate','Restricted')) then 0   
        else 4 end as L_Status_scoring,
    case when (a.application_status like 'Incomplete Application') then 1 
        when (a.application_status like 'Live Application') then 2
        when (a.application_status like 'Complete Application') then 3
        when (a.application_status like 'Declaration Required') then 4
        else 0 end as C_Status_scoring,
    case when (a.trading_status like 'NotFunded') then 6 
        when (a.trading_status like 'Funded') then 5
        else 0 end as C_TradingStatus_scoring,
    rank() over (partition by l.primary_email_address order by l.date_modified desc ) as ranks_depos  
from [SUGAR-SCA].dbo.leads l
join [SUGAR-SCA].dbo.brokers b on l.broker_id = b.id 
left join [SUGAR-SCA].dbo.accounts a on l.account_id = a.id 
join [SUGAR-SCA].dbo.users u on u.id = l.assigned_user_id 
where l.primary_email_address not like '%+%'
and l.primary_email_address not like '%test%' 
and l.date_entered > '2022-11-01'
and l.status not in ('Duplicate','Test','Restricted','NotFit')
and (a.trading_status is null or a.trading_status not in ('Active','Dormant','Inactive'))
)
,
temp1 as(
select * from temp1_reduction where temp1_reduction.ranks_depos = 1)
,
temp2_reduction as (
select  t.account_id,a.primary_email_address, a.broker_id ,t.transaction_status, max(t.date_entered) as LastDepo,
        case when (DATEDIFF(day,cast( max(t.date_entered)  as date),@dates) < 2 and  a.trading_status like 'NotFunded' and t.name like 'Deposit' and t.transaction_status like 'Rejected') then 7        
        when (DATEDIFF(day,cast( max(t.date_entered)  as date),@dates) >= 2  and  a.trading_status like 'NotFunded' and t.name like 'Deposit' and t.transaction_status like 'Rejected') then 5
        when (a.trading_status like 'Funded' and t.name like 'Deposit' and t.transaction_status like 'Rejected') then 3
        else 0 end as Upweight_Depo_Rejection,
        rank() over (partition by a.primary_email_address order by t.date_entered desc ) as ranks_status
from [SUGAR-SCA].dbo.transactions t
join [SUGAR-SCA].dbo.accounts a on a.id = t.account_id 
where t.name like 'Deposit'
and year(t.date_entered)>2021
and a.classification like 'DirectClient'
and a.primary_email_address not like '%test%'
and a.primary_email_address not like '%+%'
and t.direction like 'In'
group by t.account_id,a.primary_email_address, a.broker_id , t.name,t.transaction_status, a.trading_status,t.date_entered
),
temp2 as(
select * from temp2_reduction where temp2_reduction.ranks_status = 1
)
,
temp3 as (
select a.primary_email_address,a.broker_id, 
        max(case when (ca.name like 'Login') then ca.date_entered else 0 end) as last_login,
        case when ( datediff(day,max(case when (ca.name like 'Login') then ca.date_entered else 0 end),@dates) < 2 and 
                     count(case when (ca.name like '%assword%') then 1 else 0 end) > 2 ) then 2.5
        when (count(case when (ca.name like '%assword%') then 1 else 0 end) > 2 ) then 2
        else 0 end as Loging_PasswordUpweight,
        datediff(day,max(case when (ca.name like 'Login') then ca.date_entered else 0 end),@dates) as difflastlogin, 
        count(case when (ca.name like '%assword%') then 1 else 0 end) as psschanges 
        from [SUGAR-SCA].dbo.client_activities ca 
join [SUGAR-SCA].dbo.accounts a on a.id = ca.account_id 
where a.classification like 'DirectClient'
and a.primary_email_address not like '%test%'
and a.primary_email_address not like '%+%'
group by a.primary_email_address,a.broker_id
),
temp4_reduction as(
select COALESCE (l.primary_email_address, a.primary_email_address) as primary_email_address,  c.broker_id, c.date_entered
	from [SUGAR-SCA].dbo.calls c 
	left join [SUGAR-SCA].dbo.leads l on c.parent_id = l.id 
	left join [SUGAR-SCA].dbo.accounts a on c.parent_id = a.id 
	where year(c.date_entered)>=2022 
)
,
temp4 as(
select primary_email_address, broker_id, cast(max(date_entered) as date) as Last_call_date 
from temp4_reduction 
group by primary_email_address, broker_id)
,
temp5 as (
select temp1.primary_email_address,
    temp1.user_name,
    temp1.leadid,
    b.name,
    temp1.status,
    temp1.account_status,
    case when (temp1.account_status like 'Active') then 'Approved' else temp1.account_status end as Client_status,
--    temp1.application_status,
--    temp1.trading_status,
--    temp1.LdataCreated as Lead_created_date,
--    cast(max(temp2.LastDepo) as date) as Last_depoist_Date,
--    temp2.transaction_status,
--    cast(temp3.last_login as date) as last_login,
--    temp1.L_Status_scoring as Lead_Status_scoring,
--    temp1.C_Status_scoring as Client_Status_scoring,
--    temp1.C_TradingStatus_scoring as Trading_status_scoring,
--    temp4.Last_call_date as Last_call_date,
--    COALESCE (temp2.Upweight_Depo_Rejection,0) as Upweight_Deposit_Rejection,
--    COALESCE (temp3.Loging_PasswordUpweight,0) as Loging_Password_Upweight,
--    
--    case when (temp4.Last_call_date is not null or datediff(day,cast(max(Last_call_date) as date),@dates) >=14) then 0.1 
--    when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates) >=7 and datediff(day,cast(max(Last_call_date) as date),@dates) <14)) then 0.2  
--    when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates) >=5 and datediff(day,cast(max(Last_call_date) as date),@dates) <7)) then 0.3  
--    when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates) >=1 and datediff(day,cast(max(Last_call_date) as date),@dates) <5)) then 0.4  
--    when (temp4.Last_call_date is not null or datediff(day,cast(max(Last_call_date) as date),@dates)  <1) then 0.5  
--    else 1 end as Weight_last_call, 
    
    --sum(temp1.L_Status_scoring + temp1.C_Status_scoring + temp1.C_TradingStatus_scoring + COALESCE (temp2.Upweight_Depo_Rejection,0) + COALESCE(temp3.Loging_PasswordUpweight,0) ) as Final_score, 
    
    (sum(temp1.L_Status_scoring + temp1.C_Status_scoring + temp1.C_TradingStatus_scoring + COALESCE (temp2.Upweight_Depo_Rejection,0) + COALESCE(temp3.Loging_PasswordUpweight,0) ) *
    (case when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates) >=14)) then 0.9 -- last call more than 14 days ago
    when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates) >=7 and datediff(day,cast(max(Last_call_date) as date),@dates) <14)) then 0.8  -- last call between 7 to 14 days ago
    when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates) >=5 and datediff(day,cast(max(Last_call_date) as date),@dates) <7)) then 0.7  -- last call between 5 to 7 days ago
    when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates) >=1 and datediff(day,cast(max(Last_call_date) as date),@dates) <5)) then 0.6  -- last call between 1 to 5 days ago
    when (temp4.Last_call_date is not null or (datediff(day,cast(max(Last_call_date) as date),@dates)  <1)) then 0.5  -- last call within 24hrs
    else 1 end)) as Final_score_Call_Weighted
 
from temp1 
left join temp2 on (temp1.primary_email_address = temp2.primary_email_address and temp1.broker_id = temp2.broker_id)
left join temp3 on (temp1.primary_email_address = temp3.primary_email_address and temp1.broker_id = temp3.broker_id)
left join temp4 on (temp1.primary_email_address = temp4.primary_email_address and temp1.broker_id = temp4.broker_id)
join [SUGAR-SCA].dbo.brokers b on temp1.broker_id = b.id 

group by 
    temp1.primary_email_address,
    temp1.leadid,
    temp1.user_name,
    b.name,
    temp1.status,
    temp1.account_status,
    temp1.application_status,
    temp1.trading_status,
    temp1.LdataCreated,
    temp2.LastDepo,
    temp2.transaction_status,
    temp3.last_login,
    temp1.L_Status_scoring,
    temp1.C_Status_scoring,
    temp1.C_TradingStatus_scoring,
    temp2.Upweight_Depo_Rejection,
    temp3.Loging_PasswordUpweight,
    temp4.Last_call_date
--order by primary_email_address desc
)
select temp5.primary_email_address as Email,
    temp5.leadid,
	temp5.user_name as Name,
    temp5.name as Broker,
    temp5.status as Lead_Status,
    temp5.Client_status,
    temp5.Final_score_Call_Weighted,
    case when (temp5.Final_score_Call_Weighted > 12.999) then 'Very High'
    	when (temp5.Final_score_Call_Weighted between 8 and 12.999) then 'High'
    	when (temp5.Final_score_Call_Weighted between 5 and 7.999) then 'Average'
    	when (temp5.Final_score_Call_Weighted between 3 and 4.999) then 'Low'
    	else 'Very Low'  end as FinalScore
from temp5    
where temp5.Client_status not like 'Closed'
order by temp5.Final_score_Call_Weighted desc
        '''
ALPHA_SCA = pd.read_sql(query, cnxn)

#### Appending the JOSC & SCA in one Dataframe

In [ ]:
ALPHA_SCAJOSC = ALPHA_JOSC.append(ALPHA_SCA, ignore_index=True)

#### Aggregating Data

In [ ]:
options = ['abdulrahim.alalami','abdallah.alfayez','abdulhamid.shaikh','abdulrahim.alalami','Ahmad.Bitar','Ahmad.Qutaishat',
'ahmad.singer','ali.arif','Aline.Shahbenderian','Esam.Joudah','fadi.chamaa','faris.hadidi','filip.alahmed','haya.hussein',
'Hussein.Alchahid','joud.hajmurad','kinan.hassino','Lara.Hassan','Lodi.Shehryar','lodi.shehryar.sales','malaz.alahmad',
'mohamed.totonge','Mohammad.AbuAli','Mohammad.AbuNimreh','Mostafa.Zein','muntaser.halaseh','Nilesh.Joshi','Omar.AlKhalil',
'omar.hawasli','omar.rayyan','rama.mleihi','rami.altabesh','Rola.AbuSharaf','salim.baz','samer.baghdadi','sara.khadem',
'Sayyed.Ahsan','Wasim.Kassar','wissam.alsallakh','yaser.alhaj','yassin.mismar','younis.zakaria','zaid.abusalha','hamza.jaber']

ALPHA_SCAJOSC['CRMLink'] = np.where(ALPHA_SCAJOSC['Broker']
                                    =='Equiti UAE', 
                                    "https://crm-uae.equiti.com/index.php?module=Leads&stamp=1676748259052241500&return_module=Leads&action=DetailView&record="+ALPHA_SCAJOSC['leadid'] ,
                                    "https://honey.equitiglobalmarkets.com/#/Leads/"+ALPHA_SCAJOSC['leadid'] )

ALPHA_SCAJOSC = ALPHA_SCAJOSC.sort_values(['Name', 'Final_score_Call_Weighted'],ascending = [True, False])
ALPHA_SCAJOSC['Name2'] = ALPHA_SCAJOSC['Name'].str.strip()
ALPHA_SCAJOSC.loc[ALPHA_SCAJOSC["Name"] == "haya.hussein@equiti.com", "Name"] = 'haya.hussein'

rslt_df = ALPHA_SCAJOSC[ALPHA_SCAJOSC['Name2'].isin(options)]
# drop finale score
rslt_df = rslt_df.drop('Final_score_Call_Weighted',axis =1)
rslt_df = rslt_df.drop('Name2', axis=1)a

#### Exporting Files

In [ ]:
# Set path
path = r'C:\Users\Riccardo.Camon\OneDrive - Equiti Group Limited\ALPHA_SCORING\MENA'
path2 = r'C:\Users\Riccardo.Camon\OneDrive - Equiti Group Limited\ALPHA_SCORING\MENA'

# set date
date = datetime.now().strftime('%Y-%m-%d')

newpath = f'{path}\{date}' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
   

# Groupby 'Name' column and export different csv's with names in manager
for x,y in rslt_df.groupby('Name'):
    y.to_excel(f'{newpath}\{x}_{date}.xlsx', index = False)
    
# Back up file 
ALPHA_SCAJOSC.to_excel(f'{path2}\ALPHA_SCAJOSC_{date}.xlsx', index = False)
